In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import models
import tensorflow as tf
import numpy as np
import glob

In [2]:
tf.keras.backend.clear_session()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8886766822589783651
]


In [3]:
def read_many(directory):
    '''read all imagens in direactory
    directory: string
    
    returns list of images
    '''
    list_imgs=list(glob.glob(directory))
    imgs=[]
    for i in range(len(list_imgs)):
        try:
            img = image.load_img(list_imgs[i], target_size=(224,224,3)) #read the image in shape of 244,244,3 
            x = image.img_to_array(img)
            imgs.append(x)
        except ValueError:
            print("error reading the following image:",list_imgs[i])
    return imgs


def load_dir(directory):
    '''read images for COVID and NON-COVID
    returns:
    X - images
    y - label of the images (i.e., 0 - NON-COVID; 1 - COVID)
    '''
    
    covid=read_many("{}/COVID/*".format(directory))
    y_covid=[1]*len(covid)
    non_covid=read_many("{}/COVID/*".format(directory))
    y_non_covid=[0]*len(non_covid)
    X=np.concatenate([np.asarray(covid), np.asarray(non_covid)])
    Y=np.concatenate([np.asarray(y_covid), np.asarray(y_non_covid)])
    return (X, Y) 
    

In [4]:
X,Y=load_dir("Dataset/Train")

In [9]:
def CNN():
    '''Convolutional neural network architetures
    returns the CNN model architeture
    '''
    conv_base = ResNet50(weights='imagenet',
                         #include_top = False,
                         #input_shape = (224,224, 3)
                        )


    # conv_base.summary()
    
    conv_base.trainable = True
    #model=conv_base
    model = models.Sequential() #instance of a sequential neural network
    model.add(conv_base) #add the resnet50 to the sequential CNN
    model.add(layers.Flatten()) #add a flaterns layer
    model.add(layers.Dropout(0.2)) #add a droput layer for avoid overfiting
    model.add(layers.Dense(256, activation = 'relu')) #add relu layer for future classification
    model.add(layers.Dense(1, activation = 'sigmoid')) #add a final sigmoid layer for binary classification
    
    #model.summary()
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])  #model configuration
    
    return model

In [10]:
model=CNN()

In [11]:
model.fit(x=X/255,y=Y,epochs=10) #training the model

Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 0.6929 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 0.6906 - accuracy: 0.6429
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.6914 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 0.6930 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 0.6934 - accuracy: 0.4286
Epoch 6/10
1/1 [==============================] - 6s 6s/step - loss: 0.6954 - accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 4s 4s/step - loss: 0.6972 - accuracy: 0.5000
Epoch 8/10
1/1 [==============================] - 4s 4s/step - loss: 0.6920 - accuracy: 0.4286
Epoch 9/10
1/1 [==============================] - 4s 4s/step - loss: 0.6945 - accuracy: 0.5000
Epoch 10/10
1/1 [==============================] - 4s 4s/step - loss: 0.6955 - accuracy: 0.3571


In [13]:
model.predict(X/255) #predicting the model class

array([[0.5014292 ],
       [0.5020103 ],
       [0.502618  ],
       [0.5019128 ],
       [0.5022068 ],
       [0.5023143 ],
       [0.50172204],
       [0.5014292 ],
       [0.5020103 ],
       [0.502618  ],
       [0.5019128 ],
       [0.5022068 ],
       [0.5023143 ],
       [0.50172204]], dtype=float32)

(14,)